#### POPULARITY OF MUSIC RECORDS

The music industry has a well-developed market with a global annual revenue around $15 billion. The recording industry is highly competitive and is dominated by three big production companies which make up nearly 82% of the total annual album sales. 

Artists are at the core of the music industry and record labels provide them with the necessary resources to sell their music on a large scale. A record label incurs numerous costs (studio recording, marketing, distribution, and touring) in exchange for a percentage of the profits from album sales, singles and concert tickets.

Unfortunately, the success of an artist's release is highly uncertain: a single may be extremely popular, resulting in widespread radio play and digital downloads, while another single may turn out quite unpopular, and therefore unprofitable. 

Knowing the competitive nature of the recording industry, record labels face the fundamental decision problem of which musical releases to support to maximize their financial success. 

How can we use analytics to predict the popularity of a song? In this assignment, we challenge ourselves to predict whether a song will reach a spot in the Top 10 of the Billboard Hot 100 Chart.

Taking an analytics approach, we aim to use information about a song's properties to predict its popularity. The dataset songs.csv consists of all songs which made it to the Top 10 of the Billboard Hot 100 Chart from 1990-2010 plus a sample of additional songs that didn't make the Top 10. This data comes from three sources: Wikipedia, Billboard.com, and EchoNest.

The variables included in the dataset either describe the artist or the song, or they are associated with the following song attributes: time signature, loudness, key, pitch, tempo, and timbre.

Here's a detailed description of the variables:

- year = the year the song was released
- songtitle = the title of the song
- artistname = the name of the artist of the song
- songID and artistID = identifying variables for the song and artist
- timesignature and timesignature_confidence = a variable estimating the time signature of the song, and the confidence in the estimate
- loudness = a continuous variable indicating the average amplitude of the audio in decibels
- tempo and tempo_confidence = a variable indicating the estimated beats per minute of the song, and the confidence in the estimate
- key and key_confidence = a variable with twelve levels indicating the estimated key of the song (C, C#, . . ., B), and the confidence in the estimate
- energy = a variable that represents the overall acoustic energy of the song, using a mix of features such as loudness
- pitch = a continuous variable that indicates the pitch of the song
timbre_0_min, timbre_0_max, timbre_1_min, timbre_1_max, . . . , timbre_11_min, and timbre_11_max = variables that indicate the minimum/maximum values over all segments for each of the twelve values in the timbre vector (resulting in 24 continuous variables)
- Top10 = a binary variable indicating whether or not the song made it to the Top 10 of the Billboard Hot 100 Chart (1 if it was in the top 10, and 0 if it was not)

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.cross_validation import train_test_split, cross_val_score
# from sklearn import datasets, metrics
import sklearn.linear_model as lm

# from sklearn.tree import DecisionTreeClassifier, export_graphviz
# from sklearn.ensemble import RandomForestClassifier
# import pydot
# from os import system
# from sklearn.externals.six import StringIO
# from IPython.display import Image

#import itertools
#import pandas_datareader.data as pdweb
#from pandas_datareader.data import DataReader
#from datetime import datetime
#from io import StringIO

In [36]:
df_raw = pd.read_csv('../data/songs.csv', encoding='latin-1')
df_raw[:5]

,year,songtitle,artistname,songID,artistID,timesignature,timesignature_confidence,loudness,tempo,tempo_confidence,...,timbre_7_max,timbre_8_min,timbre_8_max,timbre_9_min,timbre_9_max,timbre_10_min,timbre_10_max,timbre_11_min,timbre_11_max,Top10
0,2010,This Is the House That Doubt Built,A Day to Remember,SOBGGAB12C5664F054,AROBSHL1187B9AFB01,3,0.853,-4.262,91.525,0.953,...,82.475,-52.025,39.116,-35.368,71.642,-126.440,18.658,-44.770,25.989,0
1,2010,Sticks & Bricks,A Day to Remember,SOPAQHU1315CD47F31,AROBSHL1187B9AFB01,4,1.000,-4.051,140.048,0.921,...,106.918,-61.320,35.378,-81.928,74.574,-103.808,121.935,-38.892,22.513,0
2,2010,All I Want,A Day to Remember,SOOIZOU1376E7C6386,AROBSHL1187B9AFB01,4,1.000,-3.571,160.512,0.489,...,80.621,-59.773,45.979,-46.293,59.904,-108.313,33.300,-43.733,25.744,0
3,2010,It's Complicated,A Day to Remember,SODRYWD1315CD49DBE,AROBSHL1187B9AFB01,4,1.000,-3.815,97.525,0.794,...,96.675,-78.660,41.088,-49.194,95.440,-102.676,46.422,-59.439,37.082,0
4,2010,2nd Sucks,A Day to Remember,SOICMQB1315CD46EE3,AROBSHL1187B9AFB01,4,0.788,-4.707,140.053,0.286,...,110.332,-56.450,37.555,-48.588,67.570,-52.796,22.888,-50.414,32.758,0


In [8]:
# How many 2010 songs are there in this dataset?

df_raw[df_raw['year']==2010].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373 entries, 0 to 372
Data columns (total 39 columns):
year                        373 non-null int64
songtitle                   373 non-null object
artistname                  373 non-null object
songID                      373 non-null object
artistID                    373 non-null object
timesignature               373 non-null int64
timesignature_confidence    373 non-null float64
loudness                    373 non-null float64
tempo                       373 non-null float64
tempo_confidence            373 non-null float64
key                         373 non-null int64
key_confidence              373 non-null float64
energy                      373 non-null float64
pitch                       373 non-null float64
timbre_0_min                373 non-null float64
timbre_0_max                373 non-null float64
timbre_1_min                373 non-null float64
timbre_1_max                373 non-null float64
timbre_2_min           

In [11]:
# How many Michael Jackson songs are there in this data set?

len(df_raw[df_raw['artistname']=='Michael Jackson'])

18

In [18]:
# How many MJ songs made top 10?

pd.crosstab(index=df_raw[df_raw['artistname']=='Michael Jackson']['artistname'],columns=df_raw['Top10'])

Top10,0,1
artistname,,
Michael Jackson,13,5


In [22]:
# Which MJ songs made top 10?

df_raw[(df_raw['artistname']=='Michael Jackson') & (df_raw['Top10']==1)]

,year,songtitle,artistname,songID,artistID,timesignature,timesignature_confidence,loudness,tempo,tempo_confidence,...,timbre_7_max,timbre_8_min,timbre_8_max,timbre_9_min,timbre_9_max,timbre_10_min,timbre_10_max,timbre_11_min,timbre_11_max,Top10
4328,2001,You Rock My World,Michael Jackson,SOBLCOF13134393021,ARXPPEY1187FB51DF4,4,1.000,-2.768,95.003,0.892,...,120.076,-53.839,63.576,-85.169,84.840,-102.185,55.266,-48.107,56.116,1
6206,1995,You Are Not Alone,Michael Jackson,SOJKNNO13737CEB162,ARXPPEY1187FB51DF4,4,1.000,-9.408,120.566,0.805,...,90.735,-61.583,60.920,-55.904,76.632,-69.799,46.173,-67.281,47.128,1
6209,1995,Black or White,Michael Jackson,SOBBRFO137756C9CB7,ARXPPEY1187FB51DF4,4,1.000,-4.017,115.027,0.535,...,107.974,-55.063,52.505,-110.999,71.477,-133.939,60.442,-55.008,43.473,1
6217,1995,Remember the Time,Michael Jackson,SOIQZMT136C9704DA5,ARXPPEY1187FB51DF4,4,1.000,-3.633,107.921,1.000,...,146.587,-58.117,62.157,-54.440,94.501,-112.348,90.437,-53.634,51.681,1
6914,1992,In The Closet,Michael Jackson,SOKIOOC12AF729ED9E,ARXPPEY1187FB51DF4,4,0.991,-4.315,110.501,0.949,...,124.354,-78.303,41.322,-83.184,106.263,-136.109,102.829,-48.192,74.575,1


In [26]:
# What are the values of timesignature that occur in our dataset?
# Which timesignature value is the most frequent among songs in our dataset?

df_raw['timesignature'].value_counts()

4    6787
3     503
1     143
5     112
7      19
0      10
Name: timesignature, dtype: int64

In [28]:
# What is the song with the highest tempo in this dataset?

df_raw[df_raw['tempo']==df_raw['tempo'].max()]

,year,songtitle,artistname,songID,artistID,timesignature,timesignature_confidence,loudness,tempo,tempo_confidence,...,timbre_7_max,timbre_8_min,timbre_8_max,timbre_9_min,timbre_9_max,timbre_10_min,timbre_10_max,timbre_11_min,timbre_11_max,Top10
6205,1995,Wanna Be Startin' Somethin',Michael Jackson,SONHIQM13738B7BE80,ARXPPEY1187FB51DF4,3,1.0,-14.528,244.307,0.566,...,93.6,-52.012,95.827,-63.554,84.129,-53.492,67.001,-73.421,67.308,0


#### Creating Our Prediction Model

We wish to predict whether or not a song will make it to the Top 10.

To do this, first use the subset function to split the data into a training set "SongsTrain" consisting of all the observations up to and including 2009 song releases, and a testing set "SongsTest", consisting of the 2010 song releases.

How many observations (songs) are in the training set?

In [38]:
SongsTrain = df_raw[df_raw['year']<=2009]
SongsTest = df_raw[df_raw['year']>2009]

len(SongsTrain)

7201

In this problem, our outcome variable is "Top10" - we are trying to predict whether or not a song will make it to the Top 10 of the Billboard Hot 100 Chart. Since the outcome variable is binary, we will build a logistic regression model. We'll start by using all song attributes as our independent variables, which we'll call Model 1.

We will only use the variables in our dataset that describe the numerical attributes of the song in our logistic regression model. So we won't use the variables "year", "songtitle", "artistname", "songID" or "artistID".

Now, use the glm function to build a logistic regression model to predict Top10 using all of the other variables as the independent variables. You should use SongsTrain to build the model.

Looking at the summary of your model, what is the value of the Akaike Information Criterion (AIC)?

In [39]:
SongsTrain.drop(["year", "songtitle", "artistname", "songID","artistID"],axis=1,inplace=True)
SongsTest.drop(["year", "songtitle", "artistname", "songID","artistID"],axis=1,inplace=True)
SongsTest.columns

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Index(['timesignature', 'timesignature_confidence', 'loudness', 'tempo',
       'tempo_confidence', 'key', 'key_confidence', 'energy', 'pitch',
       'timbre_0_min', 'timbre_0_max', 'timbre_1_min', 'timbre_1_max',
       'timbre_2_min', 'timbre_2_max', 'timbre_3_min', 'timbre_3_max',
       'timbre_4_min', 'timbre_4_max', 'timbre_5_min', 'timbre_5_max',
       'timbre_6_min', 'timbre_6_max', 'timbre_7_min', 'timbre_7_max',
       'timbre_8_min', 'timbre_8_max', 'timbre_9_min', 'timbre_9_max',
       'timbre_10_min', 'timbre_10_max', 'timbre_11_min', 'timbre_11_max',
       'Top10'],
      dtype='object')

In [58]:
# using statsmodel, so we can calculate AIC
formula = 'Top10 ~ timesignature + timesignature_confidence + loudness + tempo + tempo_confidence + key + key_confidence + energy + pitch + timbre_0_min + timbre_0_max + timbre_1_min + timbre_1_max + timbre_2_min + timbre_2_max + timbre_3_min + timbre_3_max + timbre_4_min + timbre_4_max + timbre_5_min + timbre_5_max + timbre_6_min + timbre_6_max + timbre_7_min + timbre_7_max + timbre_8_min + timbre_8_max + timbre_9_min + timbre_9_max + timbre_10_min + timbre_10_max + timbre_11_min + timbre_11_max'
LogReg0 = smf.GLM.from_formula(formula=formula, data=SongsTrain, family=sm.families.Binomial()).fit()
LogReg0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Top10   No. Observations:                 7201
Model:                            GLM   Df Residuals:                     7167
Model Family:                Binomial   Df Model:                           33
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -2379.6
Date:                Sat, 06 Aug 2016   Deviance:                       4759.2
Time:                        02:32:34   Pearson chi2:                 6.70e+03
No. Iterations:                     9                                         
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                   14.7000      1.806      8.138      0.000        11.160    18.240
timesignature                0.1264      0.087      1.457      0.145        -0.044     0.296
timesignature_confidence     0.7450      0.195      3.814      0.000         0.362     1.128
loudness                     0.2999      0.029     10.282      0.000         0.243     0.357
tempo                        0.0004      0.002      0.215      0.830        -0.003     0.004
tempo_confidence             0.4732      0.142      3.328      0.001         0.195     0.752
key                          0.0159      0.010      1.529      0.126        -0.004     0.036
key_confidence               0.3087      0.141      2.187      0.029         0.032     0.585
energy                      -1.5021      0.310     -4.847      0.000        -2.110    -0.895
pitch                      -44.9077      6.835     -6.570      0.000       -58.304   -31.511
timbre_0_min                 0.0232      0.004      5.441      0.000         0.015     0.032
timbre_0_max                -0.3310      0.026    -12.882      0.000        -0.381    -0.281
timbre_1_min                 0.0059      0.001      7.542      0.000         0.004     0.007
timbre_1_max                -0.0002      0.001     -0.342      0.732        -0.002     0.001
timbre_2_min                -0.0021      0.001     -1.889      0.059        -0.004  7.95e-05
timbre_2_max                 0.0007      0.001      0.726      0.468        -0.001     0.002
timbre_3_min                 0.0007      0.001      1.156      0.248        -0.000     0.002
timbre_3_max                -0.0030      0.001     -5.103      0.000        -0.004    -0.002
timbre_4_min                 0.0104      0.002      5.237      0.000         0.007     0.014
timbre_4_max                 0.0061      0.002      3.942      0.000         0.003     0.009
timbre_5_min                -0.0056      0.001     -4.385      0.000        -0.008    -0.003
timbre_5_max              7.736e-05      0.001      0.097      0.922        -0.001     0.002
timbre_6_min                -0.0169      0.002     -7.445      0.000        -0.021    -0.012
timbre_6_max                 0.0037      0.002      1.675      0.094        -0.001     0.008
timbre_7_min                -0.0045      0.002     -2.554      0.011        -0.008    -0.001
timbre_7_max                -0.0038      0.002     -2.060      0.039        -0.007    -0.000
timbre_8_min                 0.0039      0.003      1.372      0.170        -0.002     0.009
timbre_8_max                 0.0040      0.003      1.336      0.182        -0.002     0.010
timbre_9_min                 0.0014      0.003      0.456      0.648        -0.005     0.007
timbre_9_max                 0.0016      0.002      0.659      0.510        -0.003     0.006
timbre_10_min                0.0041      0.002      2.244      0.025         0.001     0.008


In [65]:
print(LogReg0.params)

Intercept                   14.699988
timesignature                0.126395
timesignature_confidence     0.744992
loudness                     0.299879
tempo                        0.000363
tempo_confidence             0.473227
key                          0.015882
key_confidence               0.308675
energy                      -1.502144
pitch                      -44.907740
timbre_0_min                 0.023159
timbre_0_max                -0.330982
timbre_1_min                 0.005881
timbre_1_max                -0.000245
timbre_2_min                -0.002127
timbre_2_max                 0.000659
timbre_3_min                 0.000692
timbre_3_max                -0.002967
timbre_4_min                 0.010396
timbre_4_max                 0.006111
timbre_5_min                -0.005598
timbre_5_max                 0.000077
timbre_6_min                -0.016856
timbre_6_max                 0.003668
timbre_7_min                -0.004549
timbre_7_max                -0.003774
timbre_8_min

In [68]:
# AIC is computed like this:
print ('AIC is: ',2*(33+1) - 2*LogReg0.llf)

AIC is:  4827.15410239


In [40]:
# using sklearn linear model - because Stefan says it's better

LogReg = lm.LogisticRegression()
X = SongsTrain[['timesignature', 'timesignature_confidence', 'loudness', 'tempo',
       'tempo_confidence', 'key', 'key_confidence', 'energy', 'pitch',
       'timbre_0_min', 'timbre_0_max', 'timbre_1_min', 'timbre_1_max',
       'timbre_2_min', 'timbre_2_max', 'timbre_3_min', 'timbre_3_max',
       'timbre_4_min', 'timbre_4_max', 'timbre_5_min', 'timbre_5_max',
       'timbre_6_min', 'timbre_6_max', 'timbre_7_min', 'timbre_7_max',
       'timbre_8_min', 'timbre_8_max', 'timbre_9_min', 'timbre_9_max',
       'timbre_10_min', 'timbre_10_max', 'timbre_11_min', 'timbre_11_max']]
y = SongsTrain['Top10']
LogReg.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
SongsTrain.drop('probability',axis=1,inplace=True)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [56]:
LogReg_data = SongsTrain.copy()
LogReg_data['probability'] = LogReg.predict_proba(SongsTrain.drop('Top10',axis=1)).T[1]
LogReg_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7201 entries, 373 to 7573
Data columns (total 35 columns):
timesignature               7201 non-null int64
timesignature_confidence    7201 non-null float64
loudness                    7201 non-null float64
tempo                       7201 non-null float64
tempo_confidence            7201 non-null float64
key                         7201 non-null int64
key_confidence              7201 non-null float64
energy                      7201 non-null float64
pitch                       7201 non-null float64
timbre_0_min                7201 non-null float64
timbre_0_max                7201 non-null float64
timbre_1_min                7201 non-null float64
timbre_1_max                7201 non-null float64
timbre_2_min                7201 non-null float64
timbre_2_max                7201 non-null float64
timbre_3_min                7201 non-null float64
timbre_3_max                7201 non-null float64
timbre_4_min                7201 non-null flo